In [ ]:
 # Load all certs from DER files. These will be complete certs with a reference to their signer/parent in the "issuer" field.
# Need a wrapper to support complete certs and issuers where we only have the reference to it.
import glob
import networkx as nx
from cryptography import x509
import lief

def add_to_graph(G, crt):
    # Add 
    try:
        G.add_nodes_from([crt.subject,crt.issuer])
        G.add_edge(crt.subject,crt.issuer)
    except nx.NetworkXError:
        print('Yikes!')
        print(crt.subject)
        print(crt.issuer)
    return G

certs={}
G = nx.DiGraph()
for filename in glob.glob('./pf/certs/*.crt', recursive=True):
    with open(filename, "rb") as f:
        crt = x509.load_der_x509_certificate(f.read())
        print(crt)
        add_to_graph(G,crt)
        # c2 = lief.parse(filename)  # f.read()

        # break
# G

In [ ]:
crt.issuer.public_bytes()


In [ ]:
crt.fingerprint(crt.signature_hash_algorithm).hex(":")

In [ ]:
crt.issuer.rdns

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10));

colors = []
for deg in G.out_degree:
    if deg[1]:
        colors.append("blue")
    else:
        colors.append("red")


In [ ]:
for gg in list(nx.weakly_connected_components(G)):
    g2 = G.subgraph(gg)
    nx.draw_spring(g2, node_size=5)  # , node_color=[colors[i] for i in gg])
    plt.show()

Multiple outs could be because you can issue several certificates to the same subject, i.e. if one is about to expire just give it an new one.
root cert can either be self-signed or cross-signed
for the root CA to be accepted, it's a complicated process. easier to get signed by somebody authoritative. want to be a root CA so you can issue them. x-signed usually temporary until people trust you

In [ ]:
len(list(nx.weakly_connected_components(G)))

In [ ]:
for node, deg in G.out_degree():
    if deg > 1 or deg == 0:
        print(node, deg)